In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder as ohe
from sklearn.preprocessing import StandardScaler

In [2]:
past = pd.read_csv('PastLoans.csv')
new = pd.read_csv('NewApplications_1_Round1.csv')
past.set_index(['id'], inplace=True)
new.set_index(['id'], inplace=True)

In [3]:
print(past.shape)
print(past.head())
print(new.shape)
print(new.head())

(100000, 8)
   sex  married employment   income  digital1  digital2  digital3  default
id                                                                        
1    M        0   employed  50215.0     0.213     0.193     0.032        0
2    M        0    student  14499.0     0.465     0.731     0.472        0
3    M        1    retired  32448.0     0.147     0.018     0.113        0
4    M        0    student   8627.0     0.212     0.346     0.148        0
5    F        1   employed  17259.0     0.384     0.252     0.551        0
(100000, 5)
       sex  married employment   income  digital1
id                                               
100001   M        0    student  12394.0     0.491
100002   M        0   employed  36385.0     0.450
100003   M        0    student  14945.0     0.756
100004   F        1   employed  17874.0     0.618
100005   M        0   employed  34250.0     0.453


In [4]:
past.value_counts(['default'])

default
0          81905
1          18095
dtype: int64

In [5]:
past.groupby(by=['default']).mean()

,married,income,digital1,digital2,digital3
default,,,,,
0,0.621354,40735.809010,0.392746,0.392599,0.393277
1,0.611992,23902.378281,0.298160,0.299725,0.298640


In [6]:
past1 = past.drop(['digital2', 'digital3'], axis=1)
past1.rename(columns={'digital1':'digital'}, inplace=True)
y1 = list(past1['default'])
X1 = past1.drop(['default'], axis=1)

past2 = past.drop(['digital1', 'digital3'], axis=1)
past2.rename(columns={'digital2':'digital'}, inplace=True)
y2 = list(past2['default'])
X2 = past2.drop(['default'], axis=1)

past3 = past.drop(['digital1', 'digital2'], axis=1)
past3.rename(columns={'digital3':'digital'}, inplace=True)
y3 = list(past3['default'])
X3 = past3.drop(['default'], axis=1)


In [7]:
cat_features = ['sex', 'employment']
pass_features = ['married']
numerical_features = ['income']
numerical_pass_features = ['digital']

cat_encoder = ohe()
numerical_encoder = StandardScaler()

In [8]:
X1[numerical_features] = numerical_encoder.fit_transform(X1[numerical_features])
X2[numerical_features] = numerical_encoder.fit_transform(X2[numerical_features])
X3[numerical_features] = numerical_encoder.fit_transform(X3[numerical_features])

dums = pd.get_dummies(X1[cat_features], drop_first=True)
X1.drop(cat_features, axis=1, inplace=True)
X2.drop(cat_features, axis=1, inplace=True)
X3.drop(cat_features, axis=1, inplace=True)

X1[dums.columns] = dums
X2[dums.columns] = dums
X3[dums.columns] = dums

In [9]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.2, random_state=42)

In [10]:
# cat1 = ohe().fit(X1[cat_features])
# cat2 = ohe().fit(X2[cat_features])
# cat3 = ohe().fit(X3[cat_features])

In [11]:
clf1 = LogisticRegression(random_state=42).fit(X_train1, y_train1)
clf2 = LogisticRegression(random_state=42).fit(X_train2, y_train2)
clf3 = LogisticRegression(random_state=42).fit(X_train3, y_train3)

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score


In [13]:
cross_val_score(clf1, X_train1, y_train1, scoring='f1')

array([0.01961447, 0.01359619, 0.01834862, 0.0176331 , 0.00886162])

In [15]:
f1_score(y_test1, clf1.predict(X_test1), average='macro')

0.4603174603174603

In [ ]:
clf1.score(X_train1, y_train1)

In [ ]:
clf1.predict(X_test1).sum()